Por desarrollaar

automáticamente eliminar los datos numéricos que no están en el rango

corregir columna: MDC_IDC_SET_LEADCHNL_RA_SENSING_SENSITIVITY

In [61]:
import os

import pandas as pd
from datetime import datetime

from torch.nn.functional import one_hot

DATA_INPUT_PATH = "/home/paula/Documentos/pmorenofajardo87/NOCTUAMED/noctuamed_rafa/transmisiones_a_csv/transmisiones_anonimizadas_gem/estandarizados/omop_standardized_data.csv"
CLEAN_DATA_INPUT_PATH  = "/home/paula/Documentos/pmorenofajardo87/NOCTUAMED/noctuamed_rafa/transmisiones_a_csv/data_analysis/clean_omop_standardized_data.csv"
DATA_INFO = "/home/paula/Documentos/pmorenofajardo87/NOCTUAMED/noctuamed_rafa/transmisiones_a_csv/data_analysis/omop_data_info.csv"
# columns to delete afterward
NORMALIZED_COLUMNS = []

Es este Notebook se propone un primer método de normalización de los datos

Propuestas:




In [62]:
data = pd.read_csv(CLEAN_DATA_INPUT_PATH)
omop_data_info_df = pd.read_csv(DATA_INFO)
omop_data_info_df.head()


,Unnamed: 0,Field,Data type,Expected range,Manufacturers,Sample of Values,Numbre of Values,is_date,Date_format,Date_only_one_format,...,is_categorical,is_numeric,Number of Numeric Values,Min,Max,Percentage of values within range,% are boolean,Number of Boolean Values,is_boolean,boolean_formats
0,0,MDC_IDC_SESS_DTM,Datetime,NaN,p1_abbott,"['20250311040016', '20250325060035', '20250318...",3,True,{'%Y%m%d%H%M%S'},True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,MDC_IDC_SESS_TYPE,Varchar,NaN,p1_abbott,['RemoteScheduled'],1,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,MDC_IDC_DEV_TYPE,Varchar,NaN,p1_abbott,['IPG'],1,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,MDC_IDC_DEV_MFG,Varchar,NaN,p1_abbott,['Abbott'],1,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,MDC_IDC_DEV_MODEL,Varchar,NaN,p1_abbott,['2272 Assurity MRI(TM)'],1,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### date type data



In [63]:

date_tipe_attr = omop_data_info_df[omop_data_info_df['is_date'] == True]
date_to_solve =date_tipe_attr[date_tipe_attr['Date_only_one_format'] == False]

In [64]:
date_to_solve

,Unnamed: 0,Field,Data type,Expected range,Manufacturers,Sample of Values,Numbre of Values,is_date,Date_format,Date_only_one_format,...,is_categorical,is_numeric,Number of Numeric Values,Min,Max,Percentage of values within range,% are boolean,Number of Boolean Values,is_boolean,boolean_formats
43,43,MDC_IDC_STAT_TACHYTHERAPY_RECENT_DTM_START,Datetime,Fecha válida,p1_abbott,"[20241214020524.0, 20241214020617.0, 202412140...",3,True,{'NO_MATCH'},False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,44,MDC_IDC_STAT_TACHYTHERAPY_RECENT_DTM_END,Datetime,Fecha válida,p1_abbott,"[20250311040016.0, 20250325060035.0, 202503180...",3,True,{'NO_MATCH'},False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,79,MDC_IDC_DEV_IMPLANT_DT,Date,Fecha válida,p3_boston_scientific,"['20250401', '20220603', '20210729']",3,True,{'NO_MATCH'},False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,130,MDC_IDC_SESS_DTM,Datetime,NaN,p5_microport,"['2024-09-02T02:17:49+02:00', '2024-10-08T01:0...",3,True,{'NO_MATCH'},False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
date_format =date_tipe_attr[date_tipe_attr['Date_only_one_format'] == True][['Field','Date_format']]

In [66]:
# ATENCIÓN cuento con que todas las fecchas asumen el mismo huso

def estandarizar_fecha(fecha_str: str) -> str:
    formatos = [
        "%Y%m%d%H%M%S",
        "%Y%m%dT%H%M%S%z",
        "%Y%m%dT%H%M%S",
        "%Y%m%d%H%M%S%z",
        "%Y-%m-%d"
    ]
    for fmt in formatos:
        try:
            fecha = datetime.strptime(fecha_str, fmt)
            return fecha.strftime("%Y-%m-%dT%H:%M:%SZ")  # Convertimos a UTC ISO 8601 (sin zona)
        except Exception:
            continue
    return None  # o "NO_MATCH"

In [67]:
from dateutil import parser
from datetime import datetime, timezone

def convert_to_datetime_utc(fecha_str):
    
    try:
        fecha = parser.parse(fecha_str)
        if fecha.tzinfo is None:
            # Asumir UTC si no hay zona
            return fecha.replace(tzinfo=datetime.timezone.utc)
        else:
            return fecha.astimezone(datetime.timezone.utc)
    except Exception:
        return None

In [68]:
# Convierto todos los campos date o date time a tipo ISO 8601 con zona horaria normalizada a UTC y lo guardo en {nombre del atributo}_standarize
# Convierto todos los campos date o date time a tipo datetime conservando así su zona horaria original y lo guardo en {nombre del atributo}_datetime


for date_tipe_attr in date_format['Field'].to_list():
    data[f"{date_tipe_attr}_standardize"] = data[date_tipe_attr].apply(
        lambda x: estandarizar_fecha(x) if pd.notna(x) else None
    )
    data[f"{date_tipe_attr}_datetime"] = data[date_tipe_attr].apply(
        lambda x: convert_to_datetime_utc(x) if pd.notna(x) else None
)
    NORMALIZED_COLUMNS.append(date_tipe_attr)


## Normalización de datos numéricos

In [69]:
omop_data_info_df[omop_data_info_df['is_numeric'] == True]

,Unnamed: 0,Field,Data type,Expected range,Manufacturers,Sample of Values,Numbre of Values,is_date,Date_format,Date_only_one_format,...,is_categorical,is_numeric,Number of Numeric Values,Min,Max,Percentage of values within range,% are boolean,Number of Boolean Values,is_boolean,boolean_formats
9,9,MDC_IDC_MSMT_BATTERY_REMAINING_PERCENTAGE,Numeric,0–100%,p1_abbott,"[83.0, 82.0]",2,NaN,NaN,NaN,...,NaN,True,6.0,82.00,83.00,100.0,NaN,NaN,NaN,NaN
10,10,MDC_IDC_MSMT_BATTERY_REMAINING_LONGEVITY,Numeric,NaN,p1_abbott,"[96.0, 90.0, 61.0]",3,NaN,NaN,NaN,...,NaN,True,6.0,61.00,96.00,NaN,NaN,NaN,NaN,NaN
11,11,MDC_IDC_MSMT_BATTERY_VOLTAGE,Numeric,NaN,p1_abbott,"[3.01, 2.99]",2,NaN,NaN,NaN,...,NaN,True,6.0,2.99,3.01,NaN,NaN,NaN,NaN,NaN
14,14,MDC_IDC_SET_BRADY_LOWRATE,Numeric,30–70 bpm,p1_abbott,[60.0],1,NaN,NaN,NaN,...,NaN,True,6.0,60.00,60.00,100.0,NaN,NaN,NaN,NaN
15,15,MDC_IDC_SET_BRADY_MAX_TRACKING_RATE,Numeric,NaN,p1_abbott,[125.0],1,NaN,NaN,NaN,...,NaN,True,6.0,125.00,125.00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,157,MDC_IDC_MSMT_LEADCHNL_RV_SENSING_INTR_AMPL,Numeric,0.1–20.0 mV,p5_microport,"[15.37, 6.75]",2,NaN,NaN,NaN,...,NaN,True,6.0,6.75,15.37,100.0,NaN,NaN,NaN,NaN
158,158,MDC_IDC_MSMT_LEADCHNL_RA_PACING_THRESHOLD_AMPL...,Numeric,0.25–5.0 V,p5_microport,[0.5],1,NaN,NaN,NaN,...,NaN,True,4.0,0.50,0.50,100.0,NaN,NaN,NaN,NaN
159,159,MDC_IDC_MSMT_LEADCHNL_RV_PACING_THRESHOLD_AMPL...,Numeric,0.25–5.0 V,p5_microport,"[1.0, 0.75, 0.25]",3,NaN,NaN,NaN,...,NaN,True,6.0,0.25,1.00,100.0,NaN,NaN,NaN,NaN
160,160,MDC_IDC_MSMT_LEADCHNL_RA_PACING_THRESHOLD_PULS...,Numeric,0.1–1.0 ms,p5_microport,[0.5],1,NaN,NaN,NaN,...,NaN,True,4.0,0.50,0.50,100.0,NaN,NaN,NaN,NaN


se hará una normalización de los datos numéricos según el máximo o mínimo. Se supone que antes se habrán eliminado los daos que no están en el rango

Se podría normalizar al rango en vez que a los valores min y máximo

In [70]:
from helpers.utils import normalize_columns
numeric_attr = omop_data_info_df[omop_data_info_df['is_numeric'] == True]['Field'].to_list()
data_norm = normalize_columns(data, numeric_attr, method="minmax")
NORMALIZED_COLUMNS.extend(numeric_attr)
# TODO fix MDC_IDC_SET_LEADCHNL_RA_SENSING_SENSITIVITY for abbot

Valores no normalizados en la columna 'MDC_IDC_SET_LEADCHNL_RA_SENSING_SENSITIVITY':
['AGC 0.25', 'AGC 0.25', 'AGC 0.25', 'AGC 0.25']
Valores no normalizados en la columna 'MDC_IDC_SET_LEADCHNL_RV_SENSING_SENSITIVITY':
['AGC 0.6', 'AGC 0.6', 'AGC 0.6', 'AGC 0.6']
Valores no normalizados en la columna 'MDC_IDC_SET_LEADCHNL_RA_SENSING_SENSITIVITY':
['AGC 0.25', 'AGC 0.25', 'AGC 0.25', 'AGC 0.25']
Valores no normalizados en la columna 'MDC_IDC_SET_LEADCHNL_RV_SENSING_SENSITIVITY':
['AGC 0.6', 'AGC 0.6', 'AGC 0.6', 'AGC 0.6']
Valores no normalizados en la columna 'MDC_IDC_SET_LEADCHNL_RA_SENSING_SENSITIVITY':
['AGC 0.25', 'AGC 0.25', 'AGC 0.25', 'AGC 0.25']
Valores no normalizados en la columna 'MDC_IDC_SET_LEADCHNL_RV_SENSING_SENSITIVITY':
['AGC 0.6', 'AGC 0.6', 'AGC 0.6', 'AGC 0.6']


In [71]:
data_norm[numeric_attr].sample(10)

,MDC_IDC_MSMT_BATTERY_REMAINING_PERCENTAGE,MDC_IDC_MSMT_BATTERY_REMAINING_LONGEVITY,MDC_IDC_MSMT_BATTERY_VOLTAGE,MDC_IDC_SET_BRADY_LOWRATE,MDC_IDC_SET_BRADY_MAX_TRACKING_RATE,MDC_IDC_SET_BRADY_MAX_SENSOR_RATE,MDC_IDC_SET_LEADCHNL_RA_SENSING_SENSITIVITY,MDC_IDC_SET_LEADCHNL_RV_SENSING_SENSITIVITY,MDC_IDC_SET_LEADCHNL_RA_PACING_AMPLITUDE,MDC_IDC_SET_LEADCHNL_RV_PACING_AMPLITUDE,...,MDC_IDC_SET_LEADCHNL_RA_PACING_PULSEWIDTH,MDC_IDC_SET_LEADCHNL_RV_PACING_PULSEWIDTH,MDC_IDC_MSMT_LEADCHNL_RA_IMPEDANCE_VALUE,MDC_IDC_MSMT_LEADCHNL_RV_IMPEDANCE_VALUE,MDC_IDC_MSMT_LEADCHNL_RA_SENSING_INTR_AMPL,MDC_IDC_MSMT_LEADCHNL_RV_SENSING_INTR_AMPL,MDC_IDC_MSMT_LEADCHNL_RA_PACING_THRESHOLD_AMPLITUDE,MDC_IDC_MSMT_LEADCHNL_RV_PACING_THRESHOLD_AMPLITUDE,MDC_IDC_MSMT_LEADCHNL_RA_PACING_THRESHOLD_PULSEWIDTH,MDC_IDC_MSMT_LEADCHNL_RV_PACING_THRESHOLD_PULSEWIDTH
22,NaN,94.0,2.79,50.0,120.0,120.0,0.35,2.8,1.500,2.0,...,0.4,0.40,389.0,463.0,NaN,NaN,0.500,0.750,0.4,0.40
20,NaN,1.0,2.60,60.0,130.0,120.0,0.50,2.0,1.500,2.0,...,0.4,0.40,572.0,401.0,NaN,NaN,0.500,0.625,0.4,0.40
18,59.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,100.0,NaN,NaN,40.0,130.0,130.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,90.0,NaN,NaN,60.0,NaN,NaN,NaN,NaN,NaN,1.7,...,NaN,0.40,NaN,293.0,NaN,2.60,NaN,1.200,NaN,0.40
2,82.0,90.0,2.99,60.0,125.0,115.0,0.50,2.0,1.625,1.5,...,0.4,0.40,360.0,490.0,3.10,11.70,0.625,1.250,0.4,0.40
21,NaN,1.0,2.60,60.0,130.0,120.0,0.50,2.0,1.500,2.0,...,0.4,0.40,572.0,401.0,NaN,NaN,0.500,0.625,0.4,0.40
3,82.0,90.0,2.99,60.0,125.0,115.0,0.50,2.0,1.625,1.5,...,0.4,0.40,360.0,490.0,3.10,11.70,0.625,1.250,0.4,0.40
24,NaN,NaN,3.22,40.0,NaN,NaN,NaN,0.4,NaN,2.5,...,NaN,0.35,NaN,NaN,NaN,15.37,NaN,1.000,NaN,0.35
26,NaN,NaN,3.07,50.0,135.0,135.0,0.40,0.4,3.500,3.5,...,0.5,0.50,NaN,NaN,3.22,15.37,0.500,0.750,0.5,0.50


## Normalize Categorical Data

1. One-hot encoding

Se hará one/hot encoding solo para las columnas tipo Unipolar/Bipolar que son las corregidas

In [72]:
omop_data_info_df[omop_data_info_df['Expected range'] == 'Unipolar/Bipolar']
data_categorical_attr = omop_data_info_df[omop_data_info_df['Expected range'] == 'Unipolar/Bipolar']['Field'].to_list()

In [73]:
data_norm = pd.get_dummies(data_norm, columns=data_categorical_attr, prefix_sep="_", dummy_na=False)
NORMALIZED_COLUMNS.extend(data_categorical_attr)

In [74]:
data_norm.sample(10)

,source_file,manufacturer_code,CLINIC_NAME,MANUFACTURER_PRIORITY,MDC_IDC_SESS_DTM,MDC_IDC_SESS_TYPE,MDC_IDC_DEV_TYPE,MDC_IDC_DEV_MFG,MDC_IDC_DEV_MODEL,MDC_IDC_DEV_SERIAL,...,MDC_IDC_SET_LEADCHNL_RA_PACING_POLARITY_BI,MDC_IDC_SET_LEADCHNL_RA_PACING_POLARITY_UNI,MDC_IDC_SET_LEADCHNL_RV_PACING_POLARITY_BI,MDC_IDC_SET_LEADCHNL_RV_PACING_POLARITY_UNI,MDC_IDC_SET_LEADCHNL_RA_SENSING_POLARITY_BI,MDC_IDC_SET_LEADCHNL_RV_SENSING_POLARITY_BI,MDC_IDC_SET_LEADCHNL_RA_PACING_POLARITY_BI,MDC_IDC_SET_LEADCHNL_RA_PACING_POLARITY_UNI,MDC_IDC_SET_LEADCHNL_RV_PACING_POLARITY_BI,MDC_IDC_SET_LEADCHNL_RV_PACING_POLARITY_UNI
5,p1_abbott/85204093.hl7,p1_abbott,NaN,NaN,20250318055721,RemoteScheduled,IPG,Abbott,2272 Assurity MRI(TM),6700726,...,True,False,True,False,True,True,True,False,True,False
2,p1_abbott/85204094.hl7,p1_abbott,NaN,NaN,20250325060035,RemoteScheduled,IPG,Abbott,2272 Assurity MRI(TM),6700726,...,True,False,True,False,True,True,True,False,True,False
7,p2_biotronik/02337735-94b9-4022-9a24-549b39e4c...,p2_biotronik,Hospital Universitario de Gran Canaria Dr. Negrin,Amarillo,20250327T004916+0000,Remote,IPG,Biotronik,Epyra 6 SR-T,68948906,...,False,False,False,False,False,True,False,False,False,False
28,p5_microport/1036346.xml,p5_microport,NaN,NaN,2024-10-19T00:32:08+02:00,NaN,Pacemaker,Microport,ALIZEA DR 1600,025GB06C,...,True,False,False,True,True,True,True,False,False,True
26,p5_microport/1033887.xml,p5_microport,NaN,NaN,2024-10-08T01:04:30+02:00,NaN,ICD,Microport,ULYS DF4 DR 2540,221DU181,...,True,False,True,False,True,True,True,False,True,False
17,p3_boston_scientific/1010568115.hl7,p3_boston_scientific,NaN,NaN,20250403233400+0000,NaN,DAI,Boston Scientific,VIGILANT EL ICD,619014,...,False,False,False,False,False,False,False,False,False,False
8,p2_biotronik/04f8bd87-b57c-475f-9253-23ab6dd2a...,p2_biotronik,Hospital Universitario de Gran Canaria Dr. Negrin,Amarillo,20250223T220731+0000,Remote,IPG,Biotronik,Enitra 6 SR-T,1000096403,...,False,False,False,False,False,True,False,False,False,False
6,p2_biotronik/02337735-94b9-4022-9a24-549b39e4c...,p2_biotronik,Hospital Universitario de Gran Canaria Dr. Negrin,Amarillo,20250327T004916+0000,Remote,IPG,Biotronik,Epyra 6 SR-T,68948906,...,False,False,False,False,False,True,False,False,False,False
11,p2_biotronik/04942b89-87be-4f79-a006-bf84ee70e...,p2_biotronik,Hospital Universitario de Gran Canaria Dr. Negrin,Amarillo,20250320T220247+0000,Remote,IPG,Biotronik,Enitra 6 SR-T,70402737,...,False,False,False,False,False,True,False,False,False,False
16,p3_boston_scientific/1010568115.hl7,p3_boston_scientific,NaN,NaN,20250403233400+0000,NaN,DAI,Boston Scientific,VIGILANT EL ICD,619014,...,False,False,False,False,False,False,False,False,False,False


In [75]:
data_norm.to_csv('norm_omop_standardized_data.csv', index=False)

Remove all csv data created

In [78]:
import os
for file in ['clean_omop_standardized_data.csv',
    'data_norm.csv',
    'norm_omop_standardized_data.csv',
    'OMOP_fields_merged.csv']:
    os.remove(file)